# Basic DfT REST API

- Inspo: https://www.tomforth.co.uk/toomanybuses/
- Docs: https://data.bus-data.dft.gov.uk/api/buslocation-openapi/#/SIRI-VM%20Data%20feed/get_datafeed

In [1]:
import json
import os
from pathlib import Path

import httpx
import xmltodict
from dotenv import load_dotenv
from google.transit.gtfs_realtime_pb2 import FeedMessage
from lxml import etree

In [2]:
load_dotenv()

True

## Overall settings

In [3]:
base_url = "https://data.bus-data.dft.gov.uk/api/v1/"
api_key = os.environ["API_KEY"]
adminArea = 340

## Schedules
Not actually useful to do this, easier to just bulk download the GTFS files

In [4]:
client = httpx.Client(base_url=base_url, params={"api_key": api_key})

In [5]:
r = client.get(
    "dataset/", params={"adminArea": adminArea, "limit": 500, "search": "Oxford"}
)
r.status_code

200

In [6]:
results = r.json()["results"]
len(results)

221

In [7]:
r = results[7]
r["id"], r["operatorName"], r["name"], r["lines"], r["url"]

(265,
 'Arriva UK Bus',
 'Arriva UK Bus_High Wycombe_Amersham_20200601',
 ['1', '1A', '30', '31', '32', '33', '37', '37A', '41', '48', '800', '850'],
 'https://data.bus-data.dft.gov.uk/timetable/dataset/265/download/')

In [8]:
# results[175]

In [9]:
# sorted((r["name"], i) for i, r in enumerate(results))

## Location
Rather use the bods-client Python library

In [10]:
client = httpx.Client(base_url=base_url, params={"api_key": api_key})

In [11]:
def get_activity_list(line_ref, op_ref) -> list[dict]:
    r = client.get("datafeed/", params={"lineRef": line_ref, "operatorRef": op_ref})
    assert r.status_code == 200
    d = xmltodict.parse(r.text)
    try:
        va = d["Siri"]["ServiceDelivery"]["VehicleMonitoringDelivery"]["VehicleActivity"]
        # print(json.dumps(va, indent=2))
        return list(va)
    except KeyError:
        return []

In [12]:
lines = (
    get_activity_list("5", "OXBC")
    + get_activity_list("1", "SCOX")
    + get_activity_list("10", "SCOX")
)
len(lines)

23

In [13]:
# lines[:2]

## Location GTFS

In [14]:
client = httpx.Client(base_url=base_url, params={"api_key": api_key})
r = client.get("gtfsrtdatafeed/", params={"routeId": "3815"})
assert r.status_code == 200

In [15]:
r.headers

Headers([('date', 'Sat, 15 Oct 2022 16:30:15 GMT'), ('content-type', 'application/bin; charset=utf-8'), ('content-length', '1142'), ('connection', 'keep-alive'), ('set-cookie', 'AWSALB=gOvbFeOHb8CPLmWhY+JftpVibCn2grwKqJwBprl/AlCCfJjQLGDCgIMSFkLRDZHrVb7HUKsPgwLO8RugowkAdfSUCT4XCnLXr1b/qDlGTlN7KzM1J8EimjLBa9NI; Expires=Sat, 22 Oct 2022 16:30:15 GMT; Path=/'), ('set-cookie', 'AWSALBCORS=gOvbFeOHb8CPLmWhY+JftpVibCn2grwKqJwBprl/AlCCfJjQLGDCgIMSFkLRDZHrVb7HUKsPgwLO8RugowkAdfSUCT4XCnLXr1b/qDlGTlN7KzM1J8EimjLBa9NI; Expires=Sat, 22 Oct 2022 16:30:15 GMT; Path=/; SameSite=None; Secure'), ('server', 'gunicorn/20.0.4'), ('vary', 'Accept, Cookie, Origin'), ('allow', 'GET, HEAD, OPTIONS'), ('x-frame-options', 'DENY'), ('strict-transport-security', 'max-age=31536000'), ('referrer-policy', 'same-origin')])

In [16]:
message = FeedMessage()
message.ParseFromString(r.content)

1142

In [17]:
message.entity[0]

id: "8931724906858233862"
vehicle {
  trip {
    trip_id: "VJab54fd742791dc3f118e1ed04c4259a0ae927c42"
    start_time: "15:38:00"
    start_date: "20221015"
    schedule_relationship: SCHEDULED
    route_id: "3815"
  }
  position {
    latitude: 51.7524299621582
    longitude: -1.2628819942474365
    bearing: 306.0
  }
  current_stop_sequence: 33
  current_status: IN_TRANSIT_TO
  timestamp: 1665851368
  vehicle {
    id: "SCOX-10439"
  }
}

## Load externally downloaded pb2 file

In [18]:
path = next(Path("../data").glob("*.pb2"))
message = FeedMessage()
message.ParseFromString(path.read_bytes())

3514

In [19]:
[e.vehicle.trip.route_id for e in message.entity]

['50065',
 '50065',
 '3815',
 '3815',
 '4824',
 '3815',
 '14187',
 '4824',
 '14187',
 '14187',
 '14187',
 '14187',
 '3815',
 '4824',
 '4824',
 '3815',
 '4824',
 '4824',
 '3815',
 '4824',
 '4824',
 '3815',
 '3815',
 '4824',
 '50065']